In [1]:
import $ivy.`org.apache.spark::spark-mllib:2.4.4` // Or use any other 2.x version here
import $ivy.`default::steamrec:0.1.0-SNAPSHOT`


import $ivy.$                                     // Or use any other 2.x version here

import $ivy.$                                 


In [8]:
import com.steamrec.DataLoader
import com.steamrec.User
import com.steamrec.Game
import org.apache.spark.ml.recommendation.ALS.Rating
import org.apache.spark.ml.recommendation.ALSModel
import org.apache.log4j.{Level, Logger}
import com.steamrec.ModelUtils
Logger.getLogger("org").setLevel(Level.OFF)

import com.steamrec.DataLoader

import com.steamrec.User

import com.steamrec.Game

import org.apache.spark.ml.recommendation.ALS.Rating

import org.apache.spark.ml.recommendation.ALSModel

import org.apache.log4j.{Level, Logger}

import com.steamrec.ModelUtils


In [3]:
import org.apache.spark.sql._

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}
import spark.implicits._

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@2891805
import spark.implicits._

In [4]:
val model_path = "data_dir/rank_500_it_10_lambda_0.1_alpha_1.0"

model_path: String = "data_dir/rank_500_it_10_lambda_0.1_alpha_1.0"

In [5]:
val model = ALSModel.load(s"${model_path}/model")

first at ReadWrite.scala:615

1 / 1

load at ALS.scala:527

1 / 1

load at ALS.scala:529

1 / 1

model: ALSModel = als_a0c6620a7d35

In [6]:
val data = DataLoader.loadData("data_dir/train-test-split-v2.json")

json at DataLoader.scala:11

9 / 9

collect at GameData.scala:15

9 / 9

data: com.steamrec.GameData = com.steamrec.GameData@504b9558

In [7]:
data.dataSet.head(1)

head at cmd6.sc:1

1 / 1

res6: Array[User] = Array(
  User(
    0,
    List(
      Game(0, "Dota 2 Test", 0),
      Game(1, "Gear Up", 1),
      Game(2, "Left 4 Dead 2", 9),
      Game(3, "Counter-Strike: Global Offensive", 2493),
      Game(4, "SMITE", 540),
      Game(5, "Smite - Public Test", 0),
      Game(6, "Paladins", 49656),
      Game(7, "Paladins - Public Test", 0),
      Game(8, "Team Fortress 2", 1086),
      Game(9, "Dota 2", 71638)
    ),
    List(
      Game(0, "Dota 2 Test", 0),
      Game(3, "Counter-Strike: Global Offensive", 2493),
      Game(4, "SMITE", 540),
      Game(5, "Smite - Public Test", 0),
      Game(6, "Paladins", 49656),
      Game(7, "Paladins - Public Test", 0),
      Game(8, "Team Fortress 2", 1086),
      Game(9, "Dota 2", 71638)
    ),
    List(Game(2, "Left 4 Dead 2", 9), Game(1, "Gear Up", 1))
  )
)

In [9]:
val ratings = model.recommendForAllUsers(100)

ratings: DataFrame = [user: int, recommendations: array<struct<item:int,rating:float>>]

In [10]:

import scala.collection.mutable
case class RatingItem(item: Int, rating: Float)
case class GroupedRating(user: Int, recommendations: Array[RatingItem])

def rr(user: User, ratings: Iterator[Rating[Int]]): Double = {
    val trainGames = user.train_games.map(g => g.game_id).toSet
    val validGames = user.valid_games.map(g => g.game_id).toSet
    val sortedRatings = ratings.toList
      .filter(x => !trainGames.contains(x.item))
      .sortBy(r => -r.rating);
    sortedRatings.zipWithIndex.find((r)=>  validGames contains r._1.item)
      .map(r => 1.0/ (r._2 + 1))
      .getOrElse(0.0)


}

  def mapItem(uid:Int, ratingItem:Row): Rating[Int] = {
    Rating(uid, ratingItem.getAs[Int]("item"), ratingItem.getAs[Float]("rating"))
  }

  def row2ratings(row: Row): Iterator[Rating[Int]] = {
    val uid = row.getAs[Int](0)
    row.getAs[mutable.WrappedArray[Row]]("recommendations")
      .map(x => mapItem(uid, x)).toIterator
  }

import scala.collection.mutable

defined class RatingItem
defined class GroupedRating
defined function rr
defined function mapItem
defined function row2ratings

In [11]:
model.userFactors.filter(x=>x.getAs[Int]("id") == 0).first()

first at cmd10.sc:1

1 / 1

res10: Row = [0,WrappedArray(-0.037335385, -0.008733383, 0.024352103, -0.0041547096, 0.032182526, 0.014274008, -0.038711745, 0.0044250437, -0.061623797, 0.012769637, 0.06440961, -0.042560827, -0.030923888, -0.013964174, 0.032346394, 0.02694291, -0.055387188, -0.021201564, 0.0031795627, 0.016078277, -0.0225782, -0.024403851, -0.095074296, 0.03209017, -0.0458251, 0.02693864, 0.014098344, -0.0027378493, -0.037523795, -0.005256525, 0.009141889, -0.009772336, -0.0038924874, 0.03563304, 0.020321155, -0.07970114, 0.0325638, -0.014674704, 0.03623155, 0.018706108, 0.056124996, -0.009984402, 0.06657998, -0.08080357, -0.026738599, 0.011224127, -0.1028453, 0.007832226, -0.047058403, 0.043576017, -0.05755836, -0.01138259, -0.03373226, -0.026041199, 0.032833308, 0.0406594, -4.7975953E-4, -0.0019218441, 0.0071669645, 0.012721727, 0.04898993, -0.012035682, 0.066750795, 0.022443317, -0.04937081, 0.01668032, 0.0031412523, -0.08062505, -0.038194623, 0.03315475, -0.001098953, -0.046631105, -0.0125239575, 

In [14]:
model.itemFactors.filter(x=>x.getAs[Int]("id")==9).first

first at cmd13.sc:1

1 / 1

first at cmd13.sc:1

4 / 4

res13: Row = [9,WrappedArray(-0.014655861, -0.44936153, 0.13565738, 0.182862, -0.343516, -0.08358755, -0.086097285, 0.23842578, -0.29214862, 0.2693825, 0.11983249, -0.44775778, -0.11276365, 0.19705358, 0.04782241, 0.18122227, 0.3273791, -0.0631065, -0.081560954, 0.08657331, 0.14350876, -0.3035087, 0.09299257, -0.23347463, -0.10677377, -0.39316016, 0.23610264, 0.19350655, -0.54408747, 0.0442647, -0.5524963, 0.04748626, 0.30819336, 0.39800382, 0.09351358, -0.08234835, -0.09808568, 0.30883628, -0.23198284, 0.4356096, 0.17494038, -0.47270766, 0.18076588, -0.16392532, -0.35585827, -0.17838508, -0.2991393, 0.3701254, -0.325268, 0.07660771, -0.16223335, 0.37951, 0.042944282, -0.011390528, 0.22965811, 0.34049425, -0.20766017, 0.21092941, 0.13199185, -0.19217117, 0.45475882, -0.24318531, 0.038093235, -0.09393182, -0.017096162, -0.1110273, 0.026481004, -0.32620946, 0.38089427, 0.15025218, -0.082919955, 0.027569754, 0.4565661, 0.5570141, 0.30505875, 0.091201, -0.09001812, -0.35446256, 0.032336287

In [12]:
val uRatings = ratings.filter(r=> r.getAs[Int](0)==0).head.getAs[mutable.WrappedArray[(Int, Float)]](1)

head at cmd11.sc:1

50 / 50

19/11/10 15:48:04 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
19/11/10 15:48:05 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


head at cmd11.sc:1

1 / 1

...

head at cmd11.sc:1

100 / 100

head at cmd11.sc:1

75 / 75

uRatings: mutable.WrappedArray[(Int, Float)] = WrappedArray(
  [9,0.9999958],
  [6,0.99978644],
  [3,0.99948335],
  [8,0.984784],
  [4,0.96695703],
  [5,0.48497707],
  [7,0.48088557],
  [0,0.24391495],
  [259,0.1900093],
  [120,0.16350296],
  [299,0.16275986],
  [2,0.1535278],
  [249,0.12904678],
  [164,0.12300708],
  [66,0.102884844],
  [248,0.08674241],
  [256,0.085901506],
  [438,0.08086128],
  [1169,0.07909372],
  [1589,0.07299405],
  [324,0.07008588],
  [42,0.068360016],
  [1794,0.068129204],
  [186,0.064706825],
  [1222,0.06414274],
  [170,0.06396261],
  [331,0.06188073],
  [641,0.059751403],
  [1822,0.058807362],
  [319,0.05866531],
  [938,0.05846977],
  [3256,0.058340713],
  [1670,0.05829816],
  [1002,0.05797845],
  [2938,0.057810273],
  [1800,0.057274412],
  [784,0.057258956],
  [61,0.056878213],
...

In [41]:
data.id2User.value(148).valid_games

res40: List[Game] = List(
  Game(262, "War Thunder", 0),
  Game(589, "Tom Clancy's Ghost Recon Phantoms - EU", 42),
  Game(44, "Oddworld: Abe's Oddysee", 0),
  Game(186, "Dirty Bomb", 45),
  Game(66, "Warface", 52),
  Game(955, "Source Filmmaker", 31),
  Game(1002, "Pro Evolution Soccer 2016 myClub", 8),
  Game(1140, "Tom Clancy's The Division", 341),
  Game(1141, "Tom Clancy's The Division PTS", 0)
)

In [16]:
val rrs: Dataset[(Int, Double)] = ratings.map(r => (r.getAs[Int]("user"),rr(data.id2User.value(r.getAs("user")), row2ratings(r))))

rrs: Dataset[(Int, Double)] = [_1: int, _2: double]

In [42]:
rrs.collect.map(x=>x._2).sum / rrs.count

collect at cmd41.sc:1

50 / 50

collect at cmd41.sc:1

200 / 200

count at cmd41.sc:1

...

count at cmd41.sc:1

200 / 200

count at cmd41.sc:1

1 / 1

res41: Double = 0.6315159835655693